In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile as zf
files = zf.ZipFile("/content/drive/MyDrive/BE Project/FBHM.zip", 'r')
files.extractall('memes')
files.close()

In [ ]:
pip install transformers torch torchvision tqdm
pip install torch

In [ ]:
import torch
import torchvision
print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())

In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import LxmertModel, LxmertTokenizer
LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")
LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import LxmertTokenizer, LxmertModel
from PIL import Image
from torchvision import transforms
import json
import os
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Main Script
if __name__ == "__main__":
    # Paths
    json_file = "/content/dataset22.json"
    img_dir = "/content/memes/FBHM/data/img"

    # Hyperparameters
    max_len = 128
    batch_size = 16
    learning_rate = 1e-5
    epochs = 5

    # Tokenizer and Transform
    tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    # Dataset and DataLoader
    dataset = ToxicMemeDataset(json_file, img_dir, tokenizer, max_len, transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Model, Loss, Optimizer
    model = ToxicMemeClassifier(num_classes=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Train and Evaluate
    train_model(model, train_loader, optimizer, criterion, device, epochs)
    evaluate_model(model, test_loader, device)

In [ ]:
#Dataset Loading
class ToxicMemeDataset(Dataset):
    def __init__(self, json_file, img_dir, tokenizer, max_len, transform=None):
        with open(json_file, "r") as f:
            self.data = json.load(f)
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        text = item["text"]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        img_path = os.path.join(self.img_dir, item["img"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(item["label"], dtype=torch.long)

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "image": image,
            "label": label,
        }

In [ ]:
#Model Definition - Model Architecture
class ToxicMemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ToxicMemeClassifier, self).__init__()
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

        resnet = models.resnet50(pretrained=True)
        self.cnn_backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.visual_fc = nn.Linear(resnet.fc.in_features, self.lxmert.config.visual_feat_dim)


        self.fc = nn.Linear(self.lxmert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, images):
        visual_feats = self.cnn_backbone(images).squeeze(-1).squeeze(-1)
        visual_feats = self.visual_fc(visual_feats).unsqueeze(1)

        batch_size = visual_feats.size(0)
        visual_pos = torch.zeros(batch_size, 1, 4).to(visual_feats.device)  # One region per image

        outputs = self.lxmert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            visual_feats=visual_feats,
            visual_pos=visual_pos,
        )
        logits = self.fc(outputs.pooled_output)
        return logits


In [ ]:
#Training Function
def train_model(model, data_loader, optimizer, criterion, device, epochs):
    model = model.to(device)
    for epoch in range(epochs):
        model.train()
        loop = tqdm(data_loader, desc=f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)


            outputs = model(input_ids, attention_mask, images)
            loss = criterion(outputs, labels)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        print(f"Epoch {epoch + 1}: Loss = {total_loss / len(data_loader)}")

In [ ]:
# Evaluation Function
def evaluate_model(model, data_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask, images)
            _, preds = torch.max(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {correct / total:.2f}")

In [1]:
def test_model(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            # Get model predictions
            outputs = model(input_ids, attention_mask, images)
            _, preds = torch.max(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.2f}")

    return all_preds, all_labels

In [ ]:
# Test the model on the test dataset
test_preds, test_labels = test_model(model, test_loader, device)

In [ ]:
torch.save(model.state_dict(), "toxic_meme_classifier.pth")

In [ ]:
def predict(model, img_path, text, tokenizer, transform, device):
    model = model.to(device)
    # Process text
    encoding = tokenizer(
        text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # Process image
    image = Image.open(img_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    # Model inference
    with torch.no_grad():
        output = model(input_ids, attention_mask, image)
        _, prediction = torch.max(output, dim=1)
        return prediction.item()


In [ ]:
pip install easyocr

In [ ]:
import easyocr
from PIL import Image

def extract_text_from_image(img_path):
    # Initialize the EasyOCR reader for English
    reader = easyocr.Reader(['en'])

    # Use EasyOCR to extract text from the image
    results = reader.readtext(img_path)

    # Combine all extracted text (if any)
    text = " ".join([result[1] for result in results])

    return text.strip()

# Example usage
img_path = "/content/bomb.png"
# text = extract_text_from_image(img_path)

text = "jesus"
print(f"\n Extracted Text: {text}")

# Now predict using the model
prediction = predict(model, img_path, text, tokenizer, transform, device)
print(f"Prediction: {'Toxic' if prediction == 1 else 'Non-toxic'}")
